In [92]:
from IPython import get_ipython

import numpy as np
from numpy import random, dtype
from array import array
import math
from math import floor

import matplotlib
# matplotlib.use('agg')
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib widget
# %matplotlib widget

from os import path, mkdir
from tqdm import tqdm

import ROOT
from ROOT import TCanvas, TFile, TPaveText, TMath, TF1, TH1, TH1F, TH2D, TH2F, TH3F, TRandom, TPolyMarker3D, TGraphErrors, TAxis, TGaxis, TSpline3
from ROOT import gROOT, gBenchmark, gRandom, gInterpreter, gPad, gStyle, gDirectory
from ROOT import nullptr, kBlue, kRed

# import rootpy
%jsroot on

In [93]:
ROOT.gROOT.LoadMacro('./includes/rootlogon.C')

0


 Beginning new ROOT session with private TStyle 


In [94]:
canvasCnt = 0
canvases = []

def NewCanvas(s1 = 600, s2 = 450):
    
    global canvasCnt, canvases
    n = "c_" + str(canvasCnt)
    canvases.append(TCanvas(n, n, s1, s2))
    canvases[canvasCnt].Draw()
    canvasCnt = canvasCnt+1
    return canvasCnt-1, canvases[canvasCnt-1]

In [95]:
def FillGraph(gra: TGraphErrors, x, y, ex, ey):
    gra.AddPoint(x, y)
    n = gra.GetN()
    gra.SetPointError(n-1, ex, ey)
    return n

In [96]:
run = "run530178"
out = F"out/{run}/"
exten = ".pdf"
fin = TFile(F"../data/roottople_new/{run}_new.root")
tre = fin.Get("t")

try:
    mkdir(out)
except:
    pass

mtMcp = "0.5*(tim[2]+tim[3])"
dtMcp = "tim[2]-tim[3]"
mtSipm = "0.5*(tim[1]+tim[0])"
dtSipm = "tim[1]-tim[0]"
mcSipm = "0.5*(charge[1]+charge[0])"
asymSipm = "(charge[1]-charge[0])/(charge[1]+charge[0])"
cutAnd = " && "

fidlarge = np.array([5.7, 6.7, 4.7, 5.6])
fidlarge = [5.7, 6.7, 4.7, 5.6] if run == "run530164" else fidlarge
fidlarge = [5.9, 7.0, 4.7, 5.6] if run == "run530167" else fidlarge 
fidlarge = [5.9, 7.0, 4.5, 5.5] if run == "run530181" else fidlarge
fidlarge = [5.9, 7.0, 4.5, 5.5] if run == "run530178" else fidlarge
fidLarge = F"cryPos[0]>{fidlarge[0]} && cryPos[0]<{fidlarge[1]} && cryPos[1]>{fidlarge[2]} && cryPos[1]<{fidlarge[3]}" 

Warning in <TFile::Init>: file ../data/roottople_new/run530178_new.root probably not closed, trying to recover
Info in <TFile::Recover>: ../data/roottople_new/run530178_new.root, recovered key TTree:t at address 938384542
Warning in <TFile::Init>: successfully recovered 1 keys


In [97]:
NewCanvas()
tre.Draw("charge[0]>>(600, 0, 600)", fidLarge)
cCutSipm = "charge[0] > 60 && charge[1] > 60"

In [98]:
hh = TH2D("hh", "hh", 1000, 0, 500, 1000, -1, 1)
tre.Draw("(tim[0]-tim[1])/2 : charge[0]/2+charge[1]/2 >> hh", "", "")
hh = gPad.GetPrimitive("hh")
hh.SetTitle(run)
_, cc = NewCanvas()
hh.Draw("zcol")

In [99]:
tre.Draw(F"tim[1]-tim[0]:{asymSipm}>>corrHist(1000, -.6, .6, 1000, -1, 1)", fidLarge + " && " + cCutSipm, "goff")
corrHist = gDirectory.Get("corrHist")
_, cc = NewCanvas()
cc.SetLogz(0)
corrHist.SetTitle(run)
corrHist.GetXaxis().SetTitle("(Q1-Q2)/(Q1+Q2)")
corrHist.GetYaxis().SetTitle("T1 - T2 [ns]")
corrHist.Draw("zcol")

corrProf = corrHist.ProfileX()
corrProf.Rebin(8)
corrProf.SetStats(0)
# corrProf.GetXaxis().SetRangeUser(-0.15, 0.07)
corrProf.SetMarkerSize(.2)
corrProf.SetMarkerStyle(20)
corrProf.GetXaxis().SetTitle("mean charge [pC]")
corrProf.GetYaxis().SetTitle("raw T_{diff} profile [ns]")

_, ccc = NewCanvas()
corrProf.Draw()

lin = TF1 ("lin", "[0] + [1]*x", -0.15, 0.15) # -0.035, -0.02)
# corrProf.Fit(lin, "SREMQ")

corrOffs, corrSlope = lin.GetParameter(0), lin.GetParameter(1)
print(F"{corrOffs}   {corrSlope}")


ccc.SaveAs(out+"correction"+exten)

0.0   0.0


Info in <TCanvas::Print>: pdf file out/run530178/correction.pdf has been created


In [100]:
splineCorr = TSpline3(corrProf)
corrProf.GetXaxis().SetRangeUser(-0.3, 0.3)
_, cccc = NewCanvas()
corrProf.Draw()
splineCorr.SetLineColor(kBlue)
splineCorr.Draw("sameL")

cccc.SaveAs(out+"corrSpline"+exten)

Info in <TCanvas::Print>: pdf file out/run530178/corrSpline.pdf has been created


In [106]:
histoS = TH2D("histo", "histo", 1000, 0, 500, 1000, -1, 1)
prova = TH2D("prova", "prova", 200, fidlarge[2], fidlarge[3], 100, -1, 1)
prova2 = TH2D("prova2", "prova2", 200, fidlarge[0], fidlarge[1], 200, -1, 1)

for i in tqdm(range(0, tre.GetEntries())):
    tre.GetEntry(i)
    asy = (tre.charge[1]-tre.charge[0])/(tre.charge[1]+tre.charge[0])
    val = tre.tim[0] - tre.tim[1] + splineCorr.Eval(asy)
    val *= 0.5
    mq = 0.5* (tre.charge[1]+tre.charge[0])

    if ((asy > -0.22) & (asy < 0.22)):
        histoS.Fill(mq, val)
        prova.Fill(tre.cryPos[1], val)
        prova2.Fill(tre.cryPos[0], val)

NewCanvas()
histoS.Draw("zcol")
histoS.GetXaxis().SetTitle("mean charge [pC]")
histoS.GetYaxis().SetTitle("Spline corrected t1 - t2 [ns]")



100%|██████████| 62370/62370 [00:35<00:00, 1750.03it/s]


In [107]:
NewCanvas(600,300)
# prova.GetYaxis().SetRangeUser(-0.2,0.2)
prova.Draw("zcol")
prova.GetXaxis().SetTitle("track Y [cm]")
prova.GetYaxis().SetTitle("t1 - t2 [ns]")
NewCanvas(600,300)
prova2.GetXaxis().SetTitle("track X [cm]")
prova2.GetYaxis().SetTitle("t1 - t2 [ns]")
prova2.Draw("zcol")

In [108]:
histo=histoS

In [109]:
eMin = 50
eSteps = 6
eMax = 300
steps = np.linspace(eMin, eMax, eSteps+1)
stepsNo = steps.shape[0]

fout = TFile(out + "resoCorr.root", "recreate")

gra = TGraphErrors("gra")
gra.GetXaxis().SetTitle("mean SiPM charge [pC]")
gra.GetYaxis().SetTitle("mean time cell resolution [ns]")
gra.SetTitle(run)

for i, thisEne in enumerate(steps):
    if(i+1 == stepsNo): break
    sliceName = F"slice_{i}"
    histo.GetXaxis().SetRangeUser(steps[i], steps[i+1])
    slice = histo.ProjectionY()
    fitf = TF1("fitf", "gaus", -1, 1)
    fitf.SetParameter(1, slice.GetBinCenter(slice.GetMaximumBin()))
    fitf.SetParameter(2, 0.2*slice.GetRMS())
    slice.Fit(fitf, "REMQ")
    charg = histo.ProjectionX()
    FillGraph(gra, charg.GetMean(), fitf.GetParameter(2), charg.GetRMS()/math.sqrt(charg.Integral()), fitf.GetParError(2))
    fout.cd()
    slice.Write()
    charg.Write()
    slice.Delete()

gra.Write()

_, cc = NewCanvas()
gStyle.SetOptFit(11)
gStyle.SetFitFormat("g")

# fitf = TF1("fitf", "sqrt([0]*[0]/(x*x) + [1]*[1]/x + [2]*[2])")
# fitf.SetParNames("a [ns#bulletpC]", "b [ns#bulletpC^{0.5}]", "c [ns]")

fitf = TF1("fitf", "sqrt([0]*[0]/(x*x) + [1]*[1])")
fitf.SetParNames("a [ns#bulletpC]", "b [ns]")

r = gra.Fit(fitf)

gra.SetMarkerStyle(20)
gra.SetMarkerSize(.5)

qFrom, qTo = 50, 300
scalenpe = 2.48*2
npeHeigth = 10e-3

gra.SetMinimum(npeHeigth)
gra.GetXaxis().SetRangeUser(qFrom, qTo)
fnpe = TF1("fnpe", F"x*{scalenpe}")
fnpe.SetRange(fnpe.Eval(qFrom+10), fnpe.Eval(qTo-10))
anpe = TGaxis(qFrom+10, 2*npeHeigth, qTo-10, 2*npeHeigth, "fnpe", 510, "-")
anpe.SetTitle("N_{pe}")
anpe.SetTitleOffset(-1.1)
anpe.SetLabelSize(0.03)
anpe.SetLineColor(kBlue)
anpe.SetLabelColor(kBlue)
anpe.SetTextColor(kBlue)
#anpeSetTitleSize(0.06)

gra.Draw("AP")
anpe.Draw("same")

cc.SaveAs(out + "reso_corr.pdf")
cc.SaveAs(out + "reso_corr.C")

fout.Close()

 FCN=34.4527 FROM MIGRAD    STATUS=CONVERGED      68 CALLS          69 TOTAL
                     EDM=6.34921e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  a [ns#bulletpC]   4.64661e+00   5.78422e-02   1.07120e-04   3.00806e-03
   2  b [ns]       3.39102e-02   2.96495e-04   5.48845e-07   3.83203e-01


Error in <TGraphErrors::TGraphErrors>: Cannot open file: gra, TGraphErrors is Zombie
Info in <TCanvas::Print>: pdf file out/run530178/reso_corr.pdf has been created
Info in <TCanvas::SaveSource>: C++ Macro file: out/run530178/reso_corr.C has been generated
